# Images for Presentation & Papers

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext autoreload
%autoreload 2

# find the root of the project
import os
from pathlib import Path
import polars as pl

ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
import sys

sys.path.append(str(ROOT))

## Read in the Processed Trajectory Data

In [ ]:
pl.scan_parquet(
    ROOT / "notebooks" / "clean_workflow" / "data" / "merged_trajectories.parquet"
).collect().write_database(
    table_name="merged_trajectories",
    connection="sqlite:////home/max/Development/ua-traffic-data/trajectories.db",
    if_exists="replace"
)

In [ ]:
pl.read_database_uri(
    query="SELECT * FROM merged_trajectories; WHERE vehicle_id = 1;",
    uri="sqlite:////home/max/Development/ua-traffic-data/trajectories.db",
)

In [ ]:
from datetime import timedelta


trajectory_df = (
    pl.scan_parquet(
        ROOT / "notebooks" / "clean_workflow" / "data" / "merged_trajectories.parquet"
    )
    .filter(
        pl.col("epoch_time").is_between(
            pl.col("epoch_time").min() + timedelta(hours=0, minutes=0),
            pl.col("epoch_time").min() + timedelta(hours=0, minutes=45),
        )
    )
    .with_columns(
        pl.struct(["lane", "lane_index"]).hash().alias("lane_hash"),
    )
    .collect()
)

trajectory_df.head()

### Read in the ID Mapping

In [ ]:
mapping_df = (
    pl.scan_parquet(
        ROOT.joinpath("notebooks/clean_workflow/data/vehicle_id_map.parquet")
    )
    .filter(pl.col("vehicle_id").is_in(trajectory_df["vehicle_id"]))
    .collect()
)

mapping_df.head()

### Read in the Raw Trajectory Data

In [ ]:
raw_df = (
    pl.scan_parquet(
        ROOT
        / "notebooks"
        / "clean_workflow"
        / "data"
        / "all_working_processed_1Lane.parquet"
    )
    .filter(pl.col("object_id").is_in(mapping_df["object_id"]))
    .collect()
)

## Plots! 

In [ ]:
from src.plotting.time_space import plot_time_space
from datetime import timedelta
from src.radar import Filtering


# get a 10 minute window
plot_df = (
    trajectory_df.filter(
        pl.col("epoch_time").is_between(
            trajectory_df["epoch_time"].min() + timedelta(hours=0, minutes=40),
            trajectory_df["epoch_time"].min() + timedelta(hours=0, minutes=45),
        )
        & (pl.col("lane").str.contains("WBL1"))
        & (pl.col("lane_index") == 1)
    )
    .pipe(Filtering.add_cst_timezone)
    .sort("epoch_time")
)

fig = plot_time_space(
    plot_df,
    hoverdata="vehicle_id",
    vehicle_col="vehicle_id",
    markers=True,
    s_col="s_smooth",
)

# make these plots look nice
fig.update_layout(
    title="Filtered + Joined Trajectories",
    xaxis_title="Time",
    yaxis_title="Space",
    legend_title="Vehicle ID",
    font=dict(size=18, color="black", family="Times New Roman"),
    height=800,
    width=1200,
    # # remove the background
    # plot_bgcolor="rgba(0,0,0,0)",
    # # remove the grid
    # xaxis=dict(showgrid=True, ),
    # yaxis=dict(showgrid=True),
    # # set the grid color to grey
    # xaxis_gridcolor="grey",
    # yaxis_gridcolor="grey",
    # # outline the axis in black
    # xaxis_linecolor="black",
    # yaxis_linecolor="black",
    # # make minor ticks
)


fig.show()

In [ ]:
from src.plotting.time_space import plot_time_space
from datetime import timedelta
from src.radar import Filtering


# get a 10 minute window
plot_df = (
    raw_df.filter(
        pl.col("epoch_time").is_between(
            raw_df["epoch_time"].min() + timedelta(hours=0, minutes=40),
            raw_df["epoch_time"].min() + timedelta(hours=0, minutes=45),
        )
        & (
            pl.col("object_id").is_in(
                mapping_df.filter(pl.col("vehicle_id").is_in(plot_df["vehicle_id"]))[
                    "object_id"
                ]
            )
        )
    )
    .pipe(Filtering.add_cst_timezone)
    .sort("epoch_time")
)

fig = plot_time_space(
    plot_df,
    hoverdata="object_id",
    vehicle_col="object_id",
    markers=True,
    s_col="s",
)

# make these plots look nice
fig.update_layout(
    title="Raw data",
    xaxis_title="Time",
    yaxis_title="Space",
    legend_title="Vehicle ID",
    font=dict(size=18, color="black", family="Times New Roman"),
    height=800,
    width=1200,
    # # remove the background
    # plot_bgcolor="rgba(0,0,0,0)",
    # # remove the grid
    # xaxis=dict(showgrid=True, ),
    # yaxis=dict(showgrid=True),
    # # set the grid color to grey
    # xaxis_gridcolor="grey",
    # yaxis_gridcolor="grey",
    # # outline the axis in black
    # xaxis_linecolor="black",
    # yaxis_linecolor="black",
    # # make minor ticks
)


fig.show()

In [ ]:
import plotly.graph_objects as go

veh = 1462

plot_df = trajectory_df.filter(pl.col("vehicle_id") == veh)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=plot_df["epoch_time"],
        y=plot_df["s_velocity_smooth"] * -1,
        mode="markers",
        marker=dict(color="green", size=4),
        name="Joined Trajectory",
    )
)


individual_traj = raw_df.filter(
    pl.col("object_id").is_in(
        mapping_df.filter(pl.col("vehicle_id") == veh)["object_id"]
    )
).sort("epoch_time")

for v, v_df in individual_traj.group_by("object_id"):
    fig.add_trace(
        go.Scatter(
            x=v_df["epoch_time"],
            y=v_df["f32_velocityInDir_mps"],
            mode="markers",
            marker=dict(size=4),
            name=f"{v}",
        )
    )



# make these plots look nice
fig.update_layout(
    title="Single Vehicle Trajectory",
    xaxis_title="Time",
    yaxis_title="Spped [m/s]",
    legend_title="Vehicle ID",
    font=dict(size=18, color="black", family="Times New Roman"),
    height=800,
    width=1200,
)

fig.show()

In [ ]:
7_859

In [ ]:
individual_traj